# Sanity check
## 1. Cel sprawozdania
Celem tego pliku jest dokumentacja kodu oraz przebadanie poprawności naszej symulacji poprzez porównanie z teoretycznym, spodziewanym wynikiem. 
Na końcu znajdują się wyniki analizy statystycznej po wielokrotnym uruchomieniu symulacji dla prostego przykładu.
## 2. Dokumentacja kodu

In [61]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import pandas as pd
import simpy
import seaborn
import matplotlib.pyplot as plt

Każdy obiekt w naszej symulacji jest klasą, która przechowuje informacje o sobie.

Dla pacjenta są to:
- `id`: identyfikator Pacjenta
- `arrival_time`: Godzina, o której przyszedł
- `service_start_time`: Godzina, o której wszedł do gabinetu (rozpoczęła się obsługa)
- `service_end_time`: Godzina, o której wyszedł z gabinetu (zakończenie obsługi)

Każda zmienna **`time`** jest przechowywana jako liczba minut od rozpoczęcia symulacji

In [62]:
class Pacjent:
    def __init__(self, id):
        self.day = 1
        self.id = id
        self.room = None
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None

Dla gabinetu są to:
- `id`: identyfikator gabinetu
- `env`: środowisko symulacji
- `patients_served`: liczba obsłużonych pacjentów

In [63]:
class Gabinet:
    def __init__(self, id):
        self.id = id
        self.patients_served = 0

## Parametry kliniki:
- **`number_of_ rooms`**: liczba gabinetów
- **`service_time`**: Jak długo obsługiwany jest pacjent [minuty]
- **`lambda_per_hour`**: Ile pacjentów przychodzi co godzinę (dzieli się przez 60 bo zamieniamy na minuty)
- **`sim_time`**: Długość symulacji

## Pomocnicze zmienne:
- **`queue_array`**: Kolejka (podgląd czekających pacjentów)
- **`seed`**: seed generatora liczb pseudolosowych
- **`list_rooms`**: lista gabinetów o długości **`number_of_rooms`**
- **`rooms`**: struktura Store z biblioteki simpy o wielkości **`number_of_rooms`**
- **`processed_patients`**: lista obsłużonych już pacjentów

In [64]:
class Clinic:
    def __init__(self, number_of_rooms, service_time, queue_array=[], lambda_per_hour=6/60, seed=None, sim_time=120):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []


### Metoda `czas()`:
Pomocnicza funkcja do ładnego wypisywania czasu.

Parametr `time_in_minutes`: czas w minutach

Praca zaczyna się od godziny 9:00 dlatego hours = 9+[...]

In [65]:

def czas(self, time_in_minutes):
    hours = 9+time_in_minutes//60
    minutes = time_in_minutes%60
    if len(str(int(minutes)))==1:
        return f"{int(hours)}:0{int(minutes)}"
    return f"{int(hours)}:{int(minutes)}"
Clinic.czas = czas

### Metoda `time_between_new_patients()`:
Ważna metoda zwracająca przerwę w minutach, do pojawienia się następnego pacjenta, podawana jest jako parametr metody `env.timeout()` po wygenerowaniu pacjenta.

Wykorzystuje rozkład wykładniczy o intensywności $\frac{1}{\lambda}$

In [66]:
def time_between_new_patients(self):
    if self.seed:
        np.random.seed(self.seed)
    return np.random.exponential(1/self.lambda_per_hour)
Clinic.time_between_new_patients = time_between_new_patients

### Metoda `generate_patients()`:

Metoda wygeneruje pacjenta o jakimś id i rozpocznie dla niego proces `serve_patient()`, aż do momentu {service_time} przed zamknięciem (Uznaliśmy, że pacjenci nie przychodzą np. 15 minut przed zamknięciem). Dzięki niej upłynie czas do przyjścia nastepnego pacjenta

In [67]:
def generate_patients(self):
    while True:
        if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą {service_time} przed zamknięciem
            break
        patient = Pacjent(id=self.curr_patient_id)
        patient.arrival_time = self.env.now
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} przybył do kliniki")
        self.env.process(self.serve_patient(patient))
        self.curr_patient_id += 1
        yield self.env.timeout(self.time_between_new_patients())
Clinic.generate_patients = generate_patients

### Metoda `serve_patient(patient)`

Odpalana dla jakiegoś pacjenta:
- Sprawdza czy jakiś pokój jest wolny i dodaje go do podglądu kolejki (queue_array) (Taka prawdziwa kolejka jest wbudowana w SimPy i uruchamiana jest przez `rooms.get()`)
- Jeśli mija godzina *{service_time}* od końca pracy (sim_time) wtedy pacjenci są usuwani z kolejki.
- Obsługuje pacjentów i puszcza czas dalej.

In [68]:
def serve_patient(self, patient): 
    room = yield self.rooms.get()
    self.queue_array.append(patient.id)
    if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
        yield self.rooms.put(room)
        return
    patient.service_start_time = self.env.now
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
    self.queue_array.remove(patient.id)
    yield self.env.timeout(self.service_time)

    patient.service_end_time = self.env.now
    room.patients_served += 1
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {self.czas(np.trunc(patient.arrival_time))} do {self.czas(np.trunc(patient.service_start_time))}")
    patient.room = room.id
    self.processed_patients.append(patient)
    yield self.rooms.put(room)
Clinic.serve_patient = serve_patient

### Metoda `run()` 
Uruchamia symulacje i rozpoczyna proces generowania pacjentów.

In [69]:
def run(self, day=1):
    self.env.process(self.generate_patients())
    self.env.run(until=self.sim_time)
    df = pd.DataFrame([
    {
        "day": day,
        "id": p.id,
        "room": p.room,
        "arrival_time": p.arrival_time,
        "service_start_time": p.service_start_time,
        "service_end_time": p.service_end_time,
        "waiting_time": p.service_start_time - p.arrival_time
    }
    for p in self.processed_patients
    ])
    df = df.set_index("id")
    return df
Clinic.run = run

In [70]:
def run_multiple_times(self, n):
    for i in range(1, n+1):
        print(f"-----------{i}-----------")
        self.env = simpy.Environment()
        single_df = self.run(i)
        multiple_df = pd.concat([multiple_df, single_df], ignore_index=True) if i > 1 else single_df
    return multiple_df
Clinic.run_multiple_times = run_multiple_times


## 3. Analiza statystyczna (statystyki opisowe)

In [71]:
def stats(self):
    def patient_bar_plot():
        patients_served_ls = []
        id_ls = []
        for room in self.list_rooms:
            patients_served_ls.append(room.patients_served)
            id_ls.append(str(room.id))
        fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
        fig.set_xlabel('Gabinet')
        fig.set_ylabel('Ilosc Pacjentow')
        plt.savefig("patients_served_noshow.png")
        plt.show()

    patient_bar_plot()
Clinic.stats = stats

In [72]:
clinic = Clinic(number_of_rooms=2, service_time=15)

In [73]:
clinic.run()

Czas 9:00: Pacjent 1 przybył do kliniki
Czas 9:00: Pacjent 1 wchodzi do gabinetu 1 
Czas 9:15: Pacjent 1 wychodzi z gabinetu 1, czekal od 9:00 do 9:00
Czas 9:27: Pacjent 2 przybył do kliniki
Czas 9:27: Pacjent 2 wchodzi do gabinetu 2 
Czas 9:41: Pacjent 3 przybył do kliniki
Czas 9:41: Pacjent 3 wchodzi do gabinetu 1 
Czas 9:42: Pacjent 2 wychodzi z gabinetu 2, czekal od 9:27 do 9:27
Czas 9:49: Pacjent 4 przybył do kliniki
Czas 9:49: Pacjent 4 wchodzi do gabinetu 2 
Czas 9:52: Pacjent 5 przybył do kliniki
Czas 9:56: Pacjent 3 wychodzi z gabinetu 1, czekal od 9:41 do 9:41
Czas 9:56: Pacjent 5 wchodzi do gabinetu 1 
Czas 10:04: Pacjent 4 wychodzi z gabinetu 2, czekal od 9:49 do 9:49
Czas 10:06: Pacjent 6 przybył do kliniki
Czas 10:06: Pacjent 6 wchodzi do gabinetu 2 
Czas 10:06: Pacjent 7 przybył do kliniki
Czas 10:11: Pacjent 5 wychodzi z gabinetu 1, czekal od 9:52 do 9:56
Czas 10:11: Pacjent 7 wchodzi do gabinetu 1 
Czas 10:13: Pacjent 8 przybył do kliniki
Czas 10:21: Pacjent 6 wychodzi

,day,room,arrival_time,service_start_time,service_end_time,waiting_time
id,,,,,,
1,1,1,0.000000,0.000000,15.000000,0.000000
2,1,2,27.795644,27.795644,42.795644,0.000000
3,1,1,41.292223,41.292223,56.292223,0.000000
4,1,2,49.571238,49.571238,64.571238,0.000000
5,1,1,52.678514,56.292223,71.292223,3.613709
6,1,2,66.378057,66.378057,81.378057,0.000000
7,1,1,66.575847,71.292223,86.292223,4.716376
8,1,2,73.881703,81.378057,96.378057,7.496354
9,1,1,99.398670,99.398670,114.398670,0.000000


In [74]:
clinic.stats()

In [75]:
multiple_df = clinic.run_multiple_times(3)

-----------1-----------
Czas 9:00: Pacjent 10 przybył do kliniki
Czas 9:02: Pacjent 11 przybył do kliniki
Czas 9:43: Pacjent 12 przybył do kliniki
Czas 9:57: Pacjent 13 przybył do kliniki
Czas 10:10: Pacjent 14 przybył do kliniki
Czas 10:12: Pacjent 15 przybył do kliniki
Czas 10:15: Pacjent 16 przybył do kliniki
Czas 10:16: Pacjent 17 przybył do kliniki
Czas 10:21: Pacjent 18 przybył do kliniki
-----------2-----------
Czas 9:00: Pacjent 19 przybył do kliniki
Czas 9:06: Pacjent 20 przybył do kliniki
Czas 9:17: Pacjent 21 przybył do kliniki
Czas 9:34: Pacjent 22 przybył do kliniki
Czas 9:51: Pacjent 23 przybył do kliniki
Czas 9:59: Pacjent 24 przybył do kliniki
Czas 10:07: Pacjent 25 przybył do kliniki
Czas 10:27: Pacjent 26 przybył do kliniki
-----------3-----------
Czas 9:00: Pacjent 27 przybył do kliniki
Czas 9:01: Pacjent 28 przybył do kliniki
Czas 9:09: Pacjent 29 przybył do kliniki
Czas 9:09: Pacjent 30 przybył do kliniki
Czas 9:16: Pacjent 31 przybył do kliniki
Czas 9:30: Pacjent 

In [76]:
multiple_df.describe()

,day,room,arrival_time,service_start_time,service_end_time,waiting_time
count,27.00000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2.00000,1.444444,53.063544,54.822037,69.822037,1.758493
std,0.83205,0.506370,27.481420,28.512703,28.512703,2.710401
min,1.00000,1.000000,0.000000,0.000000,15.000000,0.000000
25%,1.00000,1.000000,41.292223,41.292223,56.292223,0.000000
50%,2.00000,1.000000,52.678514,56.292223,71.292223,0.000000
75%,3.00000,2.000000,66.575847,71.292223,86.292223,3.613709
max,3.00000,2.000000,99.398670,99.398670,114.398670,7.496354
